In [5]:
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
image = cv2.imread("Test images/hand2.jpg")
hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
ycbcr=cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
b,g,r=cv2.split(image)
y,cr,cb=cv2.split(ycbcr)
h,s,v=cv2.split(hsv)
s=s/(h+s+v)
ans=((0<=h)*(h<=50)*(0.23<s)*(s<0.68)*(r>95)*(g>40)*(b>20)*(r>g)*(r>b)*(abs(r-g)>15))+((r>95)*(g>40)*(b>20)*(r>g)*(r>b)*(abs(r-g)>15)*(cr>135)*(cb>85)*(y>80)*(cr<=(1.5862*cb+20))*(cr>=(0.3448*cb+76.2069))*(cr>=(-4.5652*cb+234.5652))*(cr<=-1.15*cb+301.75)*(cr<=-2.2857*cb+432.85))
b=b*ans
g=g*ans
r=r*ans
new=cv2.merge((b,g,r))
ans=ans*255
new1=cv2.merge((ans,ans,ans))
print(type(new1))
cv2.imwrite("Test images/new1.jpeg",new1)
new1=cv2.imread("Test images/new1.jpeg")
cv2.imshow('new1',new1)
new1=255-new1
new=new+new1
cv2.imshow("new,",new)
cv2.waitKey(0)
cv2.destroyAllWindows()

/home/manavkhorasiya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
/home/manavkhorasiya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys
/home/manavkhorasiya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  
/home/manavkhorasiya/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in less
  


<class 'numpy.ndarray'>


In [ ]:
model = tf.keras.models.load_model("CNN.model")

In [ ]:
def prepare(ima):
    IMG_SIZE = 100  # 50 in txt-based
    img_array = cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    img_array=img_array/255.0  # filepathread in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1,IMG_SIZE, IMG_SIZE,1)

In [ ]:
#main function for detecting skin with background being white
def whitebg_skin(image):    
    hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
    ycbcr=cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    b,g,r=cv2.split(image)
    y,cr,cb=cv2.split(ycbcr)
    h,s,v=cv2.split(hsv)
    s=s/(h+s+v)
    ans=((0<=h)*(h<=50)*(0.23<s)*(s<0.68)*(r>95)*(g>40)*(b>20)*(r>g)*(r>b)*(abs(r-g)>15))+((r>95)*(g>40)*(b>20)*(r>g)*(r>b)*(abs(r-g)>15)*(cr>135)*(cb>85)*(y>80)*(cr<=(1.5862*cb+20))*(cr>=(0.3448*cb+76.2069))*(cr>=(-4.5652*cb+234.5652))*(cr<=-1.15*cb+301.75)*(cr<=-2.2857*cb+432.85))
    b=b*ans
    g=g*ans
    r=r*ans
    new=cv2.merge((b,g,r))
    ans=ans*255
    new1=cv2.merge((ans,ans,ans))
    cv2.imwrite("Test images/new1.jpeg",new1)
    new1=cv2.imread("Test images/new1.jpeg")
    new1=255-new1
    new=new+new1
    return new

In [ ]:
cap=cv2.VideoCapture(0)
cx=100
cy=100
rw=300
rh=300
while True:
    ret, image=cap.read()
    new=whitebg_skin(image)
    #cv2.imshow('new',new)
    pre=new[cx:rw,cy:rh]
    prediction = model.predict([prepare(pre)])
    prediction=np.argmax(prediction)
    x1=prediction
    try:
         cv2.putText(new,str(x1),(50,50),cv2.FONT_HERSHEY_SIMPLEX,1.0,(0,255,0),lineType=cv2.LINE_AA)
    except:
          print("Variable x1 is not empty")
    cv2.rectangle(new,(cx,cy),(cx+rw,cy+rh),(0,255,0),2)
    cv2.imshow('new',new)
    keypress = cv2.waitKey(1) & 0xFF
    if keypress == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()